# Gradient boosting

## Setup

In [1]:
import platform; print(platform.platform())
import sys; print("Python", sys.version)

Windows-10-10.0.19045-SP0
Python 3.10.5 (tags/v3.10.5:f377153, Jun  6 2022, 16:14:13) [MSC v.1929 64 bit (AMD64)]


In [2]:
import os
import numpy as np
import pandas as pd

from catboost import CatBoostClassifier, Pool

from sklearn.model_selection import train_test_split

In [3]:
# load data
train = pd.read_csv('../data/final/train.csv')
test = pd.read_csv('../data/final/test.csv')

In [4]:
TARGET = 'Transported'
FEATURES = [col for col in train.columns if col not in [TARGET]]

numerical = train[FEATURES].select_dtypes(include=np.number).columns.to_list()
categorical = train[FEATURES].select_dtypes(exclude=np.number).columns.to_list()

train[numerical] = train[numerical].astype(float)
train[categorical] = train[categorical].astype(str)
train[TARGET] = train[TARGET].astype(float)

test[numerical] = test[numerical].astype(float)
test[categorical] = test[categorical].astype(str)

print(f'Target: {TARGET}')
print(f'Features:\n\tnumerical: {numerical}\n\tcategorical:{categorical}')
print(f'Shapes:\n\ttrain: {train.shape}\n\ttest: {test.shape}')

train.sample(5)

Target: Transported
Features:
	numerical: ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Expenditure', 'CabinNum', 'GroupSize', 'FamilySize']
	categorical:['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'NoSpending', 'CabinDeck', 'CabinSide', 'Solo']
Shapes:
	train: (8693, 19)
	test: (4277, 18)


,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Expenditure,NoSpending,CabinDeck,CabinNum,CabinSide,GroupSize,Solo,FamilySize,Transported
3837,Earth,False,TRAPPIST-1e,23.0,False,0.0,0.0,489.0,5.0,276.0,1540.0,False,G,676.0,S,1.0,True,6.0,0.0
5413,Europa,True,TRAPPIST-1e,35.0,False,0.0,0.0,0.0,0.0,0.0,0.0,True,B,195.0,P,3.0,False,14.0,1.0
4568,Earth,False,TRAPPIST-1e,17.0,False,0.0,775.0,0.0,13.0,0.0,1576.0,False,E,304.0,P,1.0,True,6.0,1.0
5514,Earth,False,TRAPPIST-1e,55.0,False,0.0,0.0,0.0,507.0,909.0,2832.0,False,F,1123.0,S,8.0,False,16.0,1.0
7261,Earth,False,TRAPPIST-1e,24.0,False,1405.0,0.0,14.0,2.0,163.0,3168.0,False,F,1613.0,P,1.0,True,4.0,0.0


## Models

In [5]:
# data splitting
x, x_val, y, y_val = train_test_split(
    train[FEATURES],
    train[TARGET],
    test_size = 0.2,
    random_state = 42
)

# pool preparation
pool_train = Pool(x, y, cat_features=categorical)
pool_val = Pool(x_val, y_val, cat_features=categorical)

In [6]:
# default 
model = CatBoostClassifier(
    iterations = 1500,
    learning_rate=0.03,
    custom_metric = ['Accuracy', 'AUC', 'F1'],
    allow_writing_files=False
)

_ = model.fit(
    pool_train,
    eval_set = pool_val,
    verbose = 250
)

0:	learn: 0.6782815	test: 0.6783597	best: 0.6783597 (0)	total: 159ms	remaining: 3m 58s
250:	learn: 0.3662659	test: 0.3999938	best: 0.3999938 (250)	total: 7.6s	remaining: 37.8s
500:	learn: 0.3256887	test: 0.3863137	best: 0.3863137 (500)	total: 17.3s	remaining: 34.4s
750:	learn: 0.2948990	test: 0.3832214	best: 0.3830772 (739)	total: 28.7s	remaining: 28.6s
1000:	learn: 0.2704738	test: 0.3814111	best: 0.3811208 (950)	total: 36.3s	remaining: 18.1s
1250:	learn: 0.2507749	test: 0.3819523	best: 0.3811208 (950)	total: 44.4s	remaining: 8.83s
1499:	learn: 0.2330747	test: 0.3823861	best: 0.3811208 (950)	total: 50.8s	remaining: 0us

bestTest = 0.3811207533
bestIteration = 950

Shrink model to first 951 iterations.


In [7]:
# evalutation accuracy
acc = np.mean(y_val == model.predict(pool_val))
print(f'Accuracy on evaluation set {acc:.2%}')

Accuracy on evaluation set 79.93%


In [8]:
# prediction
pool_test = Pool(test[FEATURES], cat_features=categorical)
preds = model.predict(pool_test)

## Submission

In [9]:
sub = pd.read_csv('../data/raw/sample_submission.csv')
sub[TARGET] = preds.astype(bool)
sub

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False
...,...,...
4272,9266_02,True
4273,9269_01,False
4274,9271_01,True
4275,9273_01,True


In [10]:
os.makedirs('../submissions', exist_ok=True)
sub.to_csv('../submissions/catboost_default.csv', index=False)